In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))

from Datos.Data_acq import get_mnist_data
#import Data_acq as data
from models import Encoder
from models import Decoder
from models import cVAE
import matplotlib.pyplot as plt
import importlib

import importlib
import numpy as np
importlib.reload(Encoder)
importlib.reload(Decoder)
import tensorflow as tf
from keras.callbacks import EarlyStopping
importlib.reload(cVAE)
import importlib
#importlib.reload(data)
dataset="mnist"
x_train, x_test,x_val ,y_train, y_test,y_val, x_train_1, y_train_1 = get_mnist_data()


original_dim = 28*28
beta = 1.0

train_dataset = tf.data.Dataset.from_tensor_slices(((x_train, y_train), x_train))
train_dataset = train_dataset.batch(128)

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,          
    restore_best_weights=True  
)

val_dataset = tf.data.Dataset.from_tensor_slices(((x_val, y_val), x_val))
val_dataset = val_dataset.batch(128)

intermediate_dim=[64,128,256,512]
latent_dim=[2, 64, 128, 256]

#for i in range(len(intermediate_dim)):
    
for i in range(1):
    en = Encoder.build_enconder(intermediate_dim=intermediate_dim[i],latent_dim=latent_dim[i])
    de = Decoder.build_decoder(intermediate_dim=intermediate_dim[i],latent_dim=latent_dim[i])


    cvae = cVAE.CVAE(encoder=en, decoder=de, original_dim=original_dim, beta=1)
    cvae.compile(optimizer=tf.keras.optimizers.Adam())

    cvae.fit(train_dataset,
            epochs=100,
            batch_size=128,
            validation_data=val_dataset,
            callbacks=[early_stopping]
            )


    cvae.save(f"int_{intermediate_dim[i]}_lat_{latent_dim[i]}_{dataset}.keras")

Usando mnist como dataset

Epoch 1/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - kl_loss: 7.0981 - loss: 83.4694 - reconstruction_loss: 76.3713 - val_kl_loss: 2.0872 - val_loss: 42.2923 - val_reconstruction_loss: 40.2758
Epoch 2/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - kl_loss: 2.0228 - loss: 41.7705 - reconstruction_loss: 39.7477 - val_kl_loss: 1.9383 - val_loss: 38.1066 - val_reconstruction_loss: 36.2546
Epoch 3/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - kl_loss: 2.0348 - loss: 38.6165 - reconstruction_loss: 36.5817 - val_kl_loss: 2.2744 - val_loss: 36.4607 - val_reconstruction_loss: 34.2143
Epoch 4/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - kl_loss: 2.2546 - loss: 37.0498 - reconstruction_loss: 34.7952 - val_kl_loss: 2.2834 - val_loss: 35.4022 - val_reconstruction_loss: 33.1463
Epoch 5/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - kl_loss: 2.4295 - loss: 36.0922 - reconstruction_loss: 33.6626 - val_kl_loss: 2.4814 - val_loss: 34.6484 - val_reconstruction_loss: 32.203

C:\Users\galan\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


AttributeError: module 'keras._tf_keras.keras' has no attribute 'saving'

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))

from tensorflow.keras.models import load_model
from custom_layers.Sampling import  Sampling

In [ ]:
from custom_layers.Sampling  import Sampling
from models.cVAE import CVAE
modelo =  load_model("int_128_lat_2_mnist.keras",custom_objects={"CVAE": CVAE, "Sampling":Sampling})

#load_model("ruta/modelo.keras", custom_objects={"CVAE": CVAE, "Sampling": Sampling})


TypeError: <class 'models.cVAE.CVAE'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'models.cVAE', 'class_name': 'CVAE', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Custom>CVAE', 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': None, 'loss_weights': None, 'metri

In [ ]:
import os

print(os.path.exists("modelos_entreados/int_128_lat_2_mnist.keras"))


False


In [1]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.19.0
Keras version: 3.10.0
